In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 01 - Test Detection Fari\n",
    "\n",
    "Notebook per testare e visualizzare il rilevamento dei fari posteriori.\n",
    "\n",
    "**Obiettivi:**\n",
    "- Caricare un frame da video\n",
    "- Testare filtri HSV\n",
    "- Visualizzare maschere e candidati\n",
    "- Validare selezione coppia fari"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.insert(0, '..')\n",
    "\n",
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from pathlib import Path\n",
    "\n",
    "from src.detection.light_detector import LightDetector\n",
    "from src.detection.candidate_selector import CandidateSelector\n",
    "from src.utils.config_loader import load_all_configs\n",
    "\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carica Configurazione e Video"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Carica configs\n",
    "configs = load_all_configs('../config')\n",
    "detection_config = configs['detection_params']\n",
    "\n",
    "print(\"✓ Configurazione caricata\")\n",
    "print(f\"  HSV ranges: {detection_config['hsv_ranges'].keys()}\")\n",
    "print(f\"  Blob detection params: {detection_config['blob_detection']}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Carica video e prendi primo frame\n",
    "VIDEO_PATH = '../data/videos/input/video1.mp4'  # MODIFICA CON IL TUO VIDEO\n",
    "\n",
    "cap = cv2.VideoCapture(VIDEO_PATH)\n",
    "ret, frame = cap.read()\n",
    "cap.release()\n",
    "\n",
    "if not ret:\n",
    "    print(\"❌ Impossibile caricare video\")\n",
    "else:\n",
    "    print(f\"✓ Frame caricato: {frame.shape}\")\n",
    "    \n",
    "    # Mostra frame originale\n",
    "    plt.figure(figsize=(12, 8))\n",
    "    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))\n",
    "    plt.title('Frame Originale')\n",
    "    plt.axis('off')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Test Filtri HSV"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Inizializza detector\n",
    "detector = LightDetector(detection_config)\n",
    "\n",
    "# Crea maschere\n",
    "red_mask = detector.detect_red_lights(frame)\n",
    "white_mask = detector.detect_white_lights(frame)\n",
    "combined_mask = cv2.bitwise_or(red_mask, white_mask)\n",
    "\n",
    "# Applica morfologia\n",
    "cleaned_mask = detector.apply_morphology(combined_mask)\n",
    "\n",
    "print(f\"✓ Maschere create\")\n",
    "print(f\"  Pixel rossi: {np.sum(red_mask > 0)}\")\n",
    "print(f\"  Pixel bianchi: {np.sum(white_mask > 0)}\")\n",
    "print(f\"  Pixel combinati (dopo morfologia): {np.sum(cleaned_mask > 0)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualizza maschere\n",
    "fig, axes = plt.subplots(2, 3, figsize=(18, 12))\n",
    "\n",
    "# Frame originale\n",
    "axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))\n",
    "axes[0, 0].set_title('Originale')\n",
    "axes[0, 0].axis('off')\n",
    "\n",
    "# Maschera rossa\n",
    "axes[0, 1].imshow(red_mask, cmap='gray')\n",
    "axes[0, 1].set_title('Maschera Rossa')\n",
    "axes[0, 1].axis('off')\n",
    "\n",
    "# Maschera bianca\n",
    "axes[0, 2].imshow(white_mask, cmap='gray')\n",
    "axes[0, 2].set_title('Maschera Bianca')\n",
    "axes[0, 2].axis('off')\n",
    "\n",
    "# Maschera combinata\n",
    "axes[1, 0].imshow(combined_mask, cmap='gray')\n",
    "axes[1, 0].set_title('Maschera Combinata')\n",
    "axes[1, 0].axis('off')\n",
    "\n",
    "# Maschera dopo morfologia\n",
    "axes[1, 1].imshow(cleaned_mask, cmap='gray')\n",
    "axes[1, 1].set_title('Dopo Morfologia')\n",
    "axes[1, 1].axis('off')\n",
    "\n",
    "# Overlay\n",
    "overlay = frame.copy()\n",
    "overlay[cleaned_mask > 0] = [0, 255, 0]\n",
    "result = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)\n",
    "axes[1, 2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))\n",
    "axes[1, 2].set_title('Overlay su Originale')\n",
    "axes[1, 2].axis('off')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Detection e Selezione Candidati"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Rileva candidati\n",
    "candidates, mask = detector.detect_tail_lights(frame)\n",
    "\n",
    "print(f\"✓ Trovati {len(candidates)} candidati\")\n",
    "\n",
    "for i, candidate in enumerate(candidates):\n",
    "    print(f\"  Candidato {i+1}:\")\n",
    "    print(f\"    Centro: {candidate.center}\")\n",
    "    print(f\"    Area: {candidate.area:.0f} px²\")\n",
    "    print(f\"    Circolarità: {candidate.circularity:.2f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualizza candidati\n",
    "vis_frame = detector.visualize_detection(frame, candidates)\n",
    "\n",
    "plt.figure(figsize=(14, 10))\n",
    "plt.imshow(cv2.cvtColor(vis_frame, cv2.COLOR_BGR2RGB))\n",
    "plt.title(f'Candidati Rilevati ({len(candidates)} totali)')\n",
    "plt.axis('off')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Seleziona coppia migliore\n",
    "frame_width = frame.shape[1]\n",
    "selector = CandidateSelector(detection_config, frame_width)\n",
    "\n",
    "centers = selector.get_tail_light_centers(candidates)\n",
    "\n",
    "if centers:\n",
    "    print(f\"✓ Coppia selezionata:\")\n",
    "    print(f\"  Faro sinistro: {centers[0]}\")\n",
    "    print(f\"  Faro destro: {centers[1]}\")\n",
    "    print(f\"  Distanza: {abs(centers[1][0] - centers[0][0])} px\")\n",
    "    \n",
    "    # Visualizza selezione\n",
    "    result = frame.copy()\n",
    "    cv2.circle(result, centers[0], 10, (0, 255, 0), -1)\n",
    "    cv2.circle(result, centers[1], 10, (0, 255, 0), -1)\n",
    "    cv2.line(result, centers[0], centers[1], (0, 255, 0), 3)\n",
    "    \n",
    "    plt.figure(figsize=(14, 10))\n",
    "    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))\n",
    "    plt.title('Coppia Fari Selezionata')\n",
    "    plt.axis('off')\n",
    "    plt.show()\nelse:\n",
    "    print(\"❌ Nessuna coppia valida trovata\")\n",
    "    print(\"   Suggerimenti:\")\n",
    "    print(\"   - Controlla i parametri HSV\")\n",
    "    print(\"   - Verifica min/max area dei blob\")\n",
    "    print(\"   - Verifica threshold di similarità\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Tuning Parametri (Opzionale)\n",
    "\n",
    "Usa questa sezione per testare variazioni dei parametri HSV."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test con parametri custom\n",
    "custom_config = detection_config.copy()\n",
    "\n",
    "# MODIFICA QUI I PARAMETRI\n",
    "custom_config['hsv_ranges']['red']['lower1'] = [0, 80, 80]  # Esempio\n",
    "custom_config['hsv_ranges']['red']['upper1'] = [15, 255, 255]\n",
    "\n",
    "# Test\n",
    "custom_detector = LightDetector(custom_config)\n",
    "custom_candidates, custom_mask = custom_detector.detect_tail_lights(frame)\n",
    "\n",
    "print(f\"Candidati con parametri custom: {len(custom_candidates)}\")\n",
    "\n",
    "# Visualizza confronto\n",
    "fig, axes = plt.subplots(1, 2, figsize=(16, 8))\n",
    "\n",
    "vis_original = detector.visualize_detection(frame.copy(), candidates)\n",
    "vis_custom = custom_detector.visualize_detection(frame.copy(), custom_candidates)\n",
    "\n",
    "axes[0].imshow(cv2.cvtColor(vis_original, cv2.COLOR_BGR2RGB))\n",
    "axes[0].set_title(f'Originale ({len(candidates)} candidati)')\n",
    "axes[0].axis('off')\n",
    "\n",
    "axes[1].imshow(cv2.cvtColor(vis_custom, cv2.COLOR_BGR2RGB))\n",
    "axes[1].set_title(f'Custom ({len(custom_candidates)} candidati)')\n",
    "axes[1].axis('off')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Salva Configurazione Ottimizzata (Opzionale)\n",
    "\n",
    "Se trovi parametri migliori, puoi salvarli qui."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# import yaml\n",
    "# \n",
    "# # Salva custom_config\n",
    "# with open('../config/detection_params_tuned.yaml', 'w') as f:\n",
    "#     yaml.dump(custom_config, f)\n",
    "# \n",
    "# print(\"✓ Configurazione salvata\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}